# Imports

In [24]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
import os
import copy

## Game Logic

In [25]:
class Fanorona:
    def __init__(self):
        self.board = [
            [-1, -1, -1, -1, -1],
            [-1, -1, -1, -1, -1],
            [-1, 1, 0, -1, 1],
            [1, 1, 1, 1, 1],
            [1, 1, 1, 1, 1]
        ]

    def get_possible_directions(self, x, y):
        if (x % 2 == 0 and y % 2 == 0) or (x % 2 == 1 and y % 2 == 1):
            return [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]
        else:
            return [(1, 0), (-1, 0), (0, 1), (0, -1)]

    def get_possible_moves(self, player):
        player_chips = [(i, j) for i in range(5) for j in range(5) if self.board[i][j] == player]
        opponent = 1 if player == -1 else -1
        moves = {}
        moves_n = {}

        for x, y in player_chips:
            directions = self.get_possible_directions(x, y)
            for direction in directions:
                dx, dy = direction
                nx, ny = x + dx, y + dy

                if nx < 0 or nx > 4 or ny < 0 or ny > 4 or self.board[nx][ny] != 0:
                    continue

                if (x, y) not in moves_n:
                    moves_n[(x, y)] = []
                moves_n[(x, y)].append(direction)

                nnx, nny = x - dx, y - dy
                if 0 <= nnx < 5 and 0 <= nny < 5 and self.board[nnx][nny] == opponent:
                    if (x, y) not in moves:
                        moves[(x, y)] = []
                    moves[(x, y)].append(direction)
                    continue

                nnx, nny = nx + dx, ny + dy
                if 0 <= nnx < 5 and 0 <= nny < 5 and self.board[nnx][nny] == opponent:
                    if (x, y) not in moves:
                        moves[(x, y)] = []
                    moves[(x, y)].append(direction)

        return (bool(moves), moves) if moves else (False, moves_n)

    def get_possible_moves_chip(self, player, x, y, directions_before=None, player_moves=None):
        if player_moves is None:
            player_moves = []
        if directions_before is None:
            directions_before = []

        opponent = 1 if player == -1 else -1
        directions = self.get_possible_directions(x, y)
        moves = {}

        for direction in directions:
            dx, dy = direction
            nx, ny = x + dx, y + dy
            if direction in directions_before or (nx, ny) in player_moves or nx < 0 or nx > 4 or ny < 0 or ny > 4 or self.board[nx][ny] != 0:
                continue

            nnx, nny = x - dx, y - dy
            if 0 <= nnx < 5 and 0 <= nny < 5 and self.board[nnx][nny] == opponent:
                if (x, y) not in moves:
                    moves[(x, y)] = []
                moves[(x, y)].append(direction)
                continue

            nnx, nny = nx + dx, ny + dy
            if 0 <= nnx < 5 and 0 <= nny < 5 and self.board[nnx][nny] == opponent:
                if (x, y) not in moves:
                    moves[(x, y)] = []
                moves[(x, y)].append(direction)

        return moves

    def check_move(self, player, x, y, direction):
        dx, dy = direction
        opponent = 1 if player == -1 else -1

        def count_opponent_chips(nnx, nny, dx, dy):
            count = 0
            while 0 <= nnx < 5 and 0 <= nny < 5:
                if self.board[nnx][nny] == opponent:
                    count += 1
                nnx, nny = nnx + dx, nny + dy
            return count

        off_c = count_opponent_chips(x + 2 * dx, y + 2 * dy, dx, dy)
        def_c = count_opponent_chips(x - dx, y - dy, -dx, -dy)

        return off_c, def_c

    def make_move(self, player, x, y, direction, offence=False, kill=True):
        dx, dy = direction
        nx, ny = x + dx, y + dy
        self.board[nx][ny] = player
        self.board[x][y] = 0
        opponent = 1 if player == -1 else -1
        if kill:
            nnx, nny = (x + 2 * dx, y + 2 * dy) if offence else (x - dx, y - dy)
            while 0 <= nnx < 5 and 0 <= nny < 5:
                if self.board[nnx][nny] != opponent:
                    return
                self.board[nnx][nny] = 0
                nnx, nny = (nnx + dx, nny + dy) if offence else (nnx - dx, nny - dy)

## Agents

In [26]:
class NeuralNetworkAgent:
    def __init__(self, model):
        self.model = model

    def predict_move(self, board, possible_moves):
        input_board = np.array(board).flatten().reshape(1, -1)
        probabilities = self.model.predict(input_board)[0]

        valid_moves = [
            (probabilities[x * 5 * 8 + y * 8 + self.direction_to_index(direction)], (x, y), direction)
            for (x, y), directions in possible_moves.items() for direction in directions
        ]

        valid_moves.sort(reverse=True, key=lambda x: x[0])
        return valid_moves[0][1], valid_moves[0][2]

    @staticmethod
    def direction_to_index(direction):
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]
        return directions.index(direction)

In [27]:
class RandomAgent:
    def select_move(self, possible_moves):
        move_from = random.choice(list(possible_moves.keys()))
        direction = random.choice(possible_moves[move_from])
        return move_from, direction

## Game

In [28]:
def play_game(nn_model):
    fanorona = Fanorona()
    nn_agent = NeuralNetworkAgent(nn_model)
    random_agent = RandomAgent()
    player = 1
    nnWinned = False

    def play_turn(agent, player):
        canKill, possible_moves = fanorona.get_possible_moves(player)
        if not possible_moves:
            return True

        if canKill:
            move = agent.predict_move(fanorona.board, possible_moves) if player == 1 else agent.select_move(
                possible_moves)
            position, direction = move
            x, y = position
            off_c, def_c = fanorona.check_move(player, x, y, direction)
            fanorona.make_move(player, x, y, direction, offence=off_c > def_c)
            # print(f"Player {player}: {x}, {y}, {direction}, {'offence' if off_c > def_c else 'defence'}")
            positions = [position]
            directions = [direction]
            x, y = x + direction[0], y + direction[1]
            while True:
                possible_chip_moves = fanorona.get_possible_moves_chip(player, x, y, directions_before=directions, player_moves=positions)
                if not possible_chip_moves:
                    break
                move = agent.predict_move(fanorona.board, possible_chip_moves) if player == 1 else agent.select_move(
                    possible_chip_moves)
                position, direction = move
                x, y = position
                positions.append(position)
                directions.append(direction)
                off_c, def_c = fanorona.check_move(player, x, y, direction)
                fanorona.make_move(player, x, y, direction, offence=off_c > def_c)
                # print(f"Player {player}: {x}, {y}, {direction}, {'offence' if off_c > def_c else 'defence'}")
                x, y = x + direction[0], y + direction[1]
        else:
            move = agent.predict_move(fanorona.board, possible_moves) if player == 1 else agent.select_move(
                possible_moves)
            position, direction = move
            x, y = position
            fanorona.make_move(player, x, y, direction, kill=False)
            # print(f"Player {player}: {x}, {y}, {direction}")

        return False

    while True:
        if play_turn(nn_agent if player == 1 else random_agent, player):
            nnWinned = player == -1
            break
        player = -player

    return nnWinned

## Baseline

In [29]:
class DummyModel:
    def predict(self, x):
        return [np.random.rand(200)]

In [30]:
model = DummyModel()
winrate = 0
for i in range(1000):
    winrate += play_game(model)
print(winrate / 1000)

0.444


## GA

In [31]:
class GeneticAlgorithm:
    def __init__(self, population_size, num_generations, mutation_rate, crossover_rate, elite_rate, input_shape, output_size, sigma, save_path, gp):
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elite_rate = elite_rate
        self.sigma = sigma
        self.input_shape = input_shape
        self.output_size = output_size
        self.save_path = save_path
        self.gp = gp 

        if not os.path.exists(save_path):
            os.makedirs(save_path)

    def generate_population(self):
        population = []
        for _ in range(self.population_size):
            model = GeneticAlgorithm.create_model(self.input_shape, self.output_size)
            population.append((model, 0))  
        return population

    @staticmethod
    def create_model(input_shape, output_size):
        model = Sequential([
            Flatten(input_shape=input_shape),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(output_size, activation='softmax')
        ])
        return model

    @staticmethod
    def evaluate_fitness_person(model, num_games):
        nn_agent = NeuralNetworkAgent(model)
        wins = 0
        for _ in range(num_games):
            if play_game(nn_agent):
                wins += 1
        return wins / num_games

    def evaluate_fitness(self, population, num_games):
        fitness_scores = []
        for model, _ in population:
            fitness_scores.append(GeneticAlgorithm.evaluate_fitness_person(model, num_games))
        return fitness_scores

    def gaussian_mutation(self, model, sigma=0.1):
        model_copy = tf.keras.models.clone_model(model)
        model_copy.set_weights(copy.deepcopy(model.get_weights()))
        for layer in model_copy.layers:
            weights = layer.get_weights()
            for i in range(len(weights)):
                weights[i] += np.random.normal(0, sigma, weights[i].shape)
            layer.set_weights(weights)
        return model_copy

    def adjust_sigma(self, generation, max_generations, initial_sigma=0.1, final_sigma=0.01):
        return initial_sigma - (generation / max_generations) * (initial_sigma - final_sigma)

    def mutate_parents(self, parents, sigma):
        mutated_parents = []
        for parent in parents:
            mutated_parent = self.gaussian_mutation(parent, sigma)
            mutated_parents.append(mutated_parent)
        return mutated_parents

    def crossover_parents(self, parents, fitnesses):
        
        dataset = [(copy.deepcopy(p1.get_weights()), copy.deepcopy(p2.get_weights())) for p1, p2 in parents]
        parent_fitnesses = fitnesses

        best_crossover_tree = self.gp.gp(dataset, parent_fitnesses)

        offspring = []
        for (p1_weights, p2_weights) in dataset:
            raw_offspring_weights = self.gp.evaluate_tree(best_crossover_tree, p1_weights, p2_weights)

            model = GeneticAlgorithm.create_model(self.input_shape, self.output_size)
            model_weights_shapes = [w.shape for w in model.get_weights()]

            reshaped_weights_shapes = [w.shape for w in raw_offspring_weights]
            if model_weights_shapes != reshaped_weights_shapes:
                raise ValueError(f"Shape mismatch after reshaping: Model expects {model_weights_shapes}, but got {reshaped_weights_shapes}")

            model.set_weights(raw_offspring_weights)
            offspring.append((model, 0))

        return offspring

    def select(self, population, rate, fitness, tournament_size=2):
        selected = []
        for _ in range(rate):
            tournament_indices = random.sample(range(len(population)), tournament_size)
            best_index = max(tournament_indices, key=lambda idx: fitness[idx])
            selected.append(population[best_index][0])
        return selected

    def save_weights(self, population, generation):
        for idx, (model, _) in enumerate(population):
            model.save_weights(os.path.join(self.save_path, f'weights_generation_{generation}_model_{idx}.h5'))

    def load_weights(self, population, generation):
        for idx, (model, _) in enumerate(population):
            model.load_weights(os.path.join(self.save_path, f'weights_generation_{generation}_model_{idx}.h5'))

    def ga(self):
        population = self.generate_population()

        for generation in range(self.num_generations):
            fitness_scores = self.evaluate_fitness(population, num_games=10)

            mutation_parents = self.select(population, int(self.mutation_rate * self.population_size), fitness_scores)
            mutated_parents = self.mutate_parents(mutation_parents, sigma=self.sigma)

            crossover_parents_indices = self.select(population, int(self.crossover_rate * self.population_size * 2), fitness_scores)
            crossover_parents = [(crossover_parents_indices[i], crossover_parents_indices[i + 1]) for i in range(0, len(crossover_parents_indices), 2)]
            crossover_fitnesses = [(fitness_scores[i], fitness_scores[i + 1]) for i in range(0, len(crossover_parents_indices), 2)]

            crossed_offsprings = self.crossover_parents(crossover_parents, crossover_fitnesses)

            elites = self.select(population, int(self.elite_rate * self.population_size), fitness_scores)

            population = elites + crossed_offsprings + mutated_parents + self.select(
                population,
                int((1 - self.mutation_rate - self.crossover_rate - self.elite_rate) * self.population_size),
                fitness_scores
            )
            self.sigma = self.adjust_sigma(generation, self.num_generations)

            self.save_weights(population, generation)

## GP

In [32]:
class GP:
    def __init__(self, max_tree_depth, population_size, num_generations, mutation_rate,
                 crossover_rate, elite_rate, input_shape, output_size):
        self.parent_fitnesses = []
        self.dataset = []
        self.max_tree_depth = max_tree_depth
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elite_rate = elite_rate
        self.input_shape = input_shape
        self.output_size = output_size
        self.function_set = ["Sum", "Minus", "Mean", "1Point", "+", "-", "*"]
        self.terminal_set = ["W1", "W2"]

    def grow_tree(self, depth):
        if depth == 0 or (random.random() < 0.3 and depth != self.max_tree_depth):
            return random.choice(self.terminal_set)
        else:
            func = random.choice(self.function_set)
            if func in ["Sum", "Minus", "Mean", "1Point"]:
                return [func, self.grow_tree(depth - 1), self.grow_tree(depth - 1)]
            elif func in ["+", "-", "*"]:
                return [func, self.grow_tree(depth - 1), random.choice([1, 2, 0.5])]

    def evaluate_tree(self, tree, W1, W2):
        if tree == "W1":
            return [np.copy(w) for w in W1]
        if tree == "W2":
            return [np.copy(w) for w in W2]

        func = tree[0]
        if func == "Sum":
            return [w1 + w2 for w1, w2 in zip(self.evaluate_tree(tree[1], W1, W2), self.evaluate_tree(tree[2], W1, W2))]
        elif func == "Minus":
            return [w1 - w2 for w1, w2 in zip(self.evaluate_tree(tree[1], W1, W2), self.evaluate_tree(tree[2], W1, W2))]
        elif func == "Mean":
            return [(w1 + w2) / 2 for w1, w2 in
                    zip(self.evaluate_tree(tree[1], W1, W2), self.evaluate_tree(tree[2], W1, W2))]
        elif func == "1Point":
            result = []
            for w1, w2 in zip(self.evaluate_tree(tree[1], W1, W2), self.evaluate_tree(tree[2], W1, W2)):
                if w1.size <= 1 or w2.size <= 1:
                    result.append(np.copy(w1))
                    continue
                point = np.random.randint(1, w1.size)
                w1_flat = w1.flatten()
                w2_flat = w2.flatten()
                combined = np.concatenate([w1_flat[:point], w2_flat[point:]])
                result.append(combined.reshape(w1.shape))
            return result
        elif func == "+":
            return [w + tree[2] for w in self.evaluate_tree(tree[1], W1, W2)]
        elif func == "-":
            return [w - tree[2] for w in self.evaluate_tree(tree[1], W1, W2)]
        elif func == "*":
            return [w * tree[2] for w in self.evaluate_tree(tree[1], W1, W2)]

    def fitness_data(self, parent1_fitness, parent2_fitness, child_weights, num_games=10):
        child_model = GeneticAlgorithm.create_model(self.input_shape, self.output_size)

        model_weights_shapes = [w.shape for w in child_model.get_weights()]

        reshaped_weights_shapes = [w.shape for w in child_weights]
        if model_weights_shapes != reshaped_weights_shapes:
            raise ValueError(
                f"Shape mismatch after reshaping: Model expects {model_weights_shapes}, but got {reshaped_weights_shapes}")

        child_model.set_weights(child_weights)
        child_fitness = GeneticAlgorithm.evaluate_fitness_person(child_model, num_games)
        return (parent1_fitness + parent2_fitness) / (2 * child_fitness)

    def fitness(self, tree):
        total_fitness = 0
        for data, fitness_pair in zip(self.dataset, self.parent_fitnesses):
            W1, W2 = data
            F1, F2 = fitness_pair
            child_weights = self.evaluate_tree(tree, W1, W2)
            total_fitness += self.fitness_data(F1, F2, child_weights)
        return total_fitness / len(self.dataset)

    def generate_population(self):
        return [self.grow_tree(self.max_tree_depth) for _ in range(self.population_size)]

    def select(self, population, rate, fitness, tournament_size=2):
        selected = []
        for _ in range(rate):
            tournament_indices = random.sample(range(len(population)), tournament_size)
            best_index = max(tournament_indices, key=lambda idx: fitness[idx])
            selected.append(population[best_index])
        return selected

    def mutate_tree(self, treeR):
        tree = copy.deepcopy(treeR)

        def mutate_subtree(subtree, depth):
            if isinstance(subtree, list) or subtree in self.terminal_set:
                if random.random() < 0.5 or depth == 0 or subtree in self.terminal_set:
                    return self.grow_tree(self.max_tree_depth - depth)
                else:
                    index = random.randint(1, len(subtree) - 1)
                    subtree[index] = mutate_subtree(subtree[index], depth - 1)
            else:
                if isinstance(subtree, (int, float)):
                    return random.choice([1, 2, 0.5])
            return subtree

        return mutate_subtree(tree, self.max_tree_depth)

    def crossover_trees(self, parent1root, parent2root, max_attempts=10):
        parent1 = copy.deepcopy(parent1root)
        parent2 = copy.deepcopy(parent2root)

        def find_all_subtrees(tree, depth, exclude_root=True):
            subtrees = []
            if isinstance(tree, list) and depth > 0:
                for i in range(1 if exclude_root else 0, len(tree)):
                    child = tree[i]
                    if isinstance(child, list):
                        func = child[0]
                        if func in ["Sum", "Minus", "Mean", "1Point", "+", "-", "*"]:
                            subtrees.append((child, tree, i, "weight"))
                        subtrees.extend(find_all_subtrees(child, depth - 1, exclude_root=False))
                    elif child in self.terminal_set:
                        subtrees.append((child, tree, i, "weight"))
                    elif isinstance(child, (int, float)):
                        subtrees.append((child, tree, i, "constant"))
            return subtrees

        for _ in range(max_attempts):
            subtrees1 = find_all_subtrees(parent1, self.max_tree_depth)
            if not subtrees1:
                continue

            subtree1, parent1_ref, index1, type1 = random.choice(subtrees1)

            subtrees2 = find_all_subtrees(parent2, self.max_tree_depth)
            matching_subtrees2 = [(subtree, parent, index) for subtree, parent, index, type2 in subtrees2 if
                                  type2 == type1]
            if not matching_subtrees2:
                continue

            subtree2, parent2_ref, index2 = random.choice(matching_subtrees2)

            parent1_ref[index1], parent2_ref[index2] = parent2_ref[index2], parent1_ref[index1]
            return parent1, parent2

        return parent1, parent2

    def create_crossover_operator(self, population):
        fitness_scores = [self.fitness(tree) for tree in population]
        best_index = np.argmax(fitness_scores)
        return population[best_index]

    def gp(self, dataset, parent_fitnesses):
        self.dataset = dataset
        self.parent_fitnesses = parent_fitnesses
        population = self.generate_population()
        for generation in range(self.num_generations):
            fitness_scores = [self.fitness(tree) for tree in population]

            mutation_parents = self.select(population, int(self.mutation_rate * self.population_size), fitness_scores)
            mutated_parents = [self.mutate_tree(parent) for parent in mutation_parents]

            crossover_parents = self.select(population, int(self.crossover_rate * self.population_size), fitness_scores)
            crossed_offsprings = []
            for parent1, parent2 in zip(crossover_parents[::2], crossover_parents[1::2]):
                crossed_offsprings.extend(self.crossover_trees(parent1, parent2))

            elites = self.select(population, int(self.elite_rate * self.population_size), fitness_scores)
            population = elites + crossed_offsprings + mutated_parents + self.select(
                population,
                int((1 - self.mutation_rate - self.crossover_rate - self.elite_rate) * self.population_size),
                fitness_scores
            )

        return self.create_crossover_operator(population)

## GP testing

In [33]:
import numpy as np
import random


class GeneticAlgorithm:
    @staticmethod
    def create_model(input_shape, output_size):
        return MockModel(input_shape, output_size)

    @staticmethod
    def evaluate_fitness_person(weights, num_games):
        return random.uniform(1, 10)


class MockModel:
    def __init__(self, input_shape, output_size):
        self.input_shape = input_shape
        self.output_size = output_size
        self.weights = [] 

    def set_weights(self, weights):
        self.weights = weights 

    def get_weights(self):
        if not self.weights:
            self.weights = [
                np.random.rand(2, 4),  
                np.random.rand(4),  
                np.random.rand(4, 3),  
                np.random.rand(3),
                np.random.rand(3, 1),
                np.random.rand(1)  
            ]
        return self.weights


weights1 = [
    np.array([[0.1, 0.2, 0.3, 0.4],
              [0.5, 0.6, 0.7, 0.8]]),
    np.array([0.1, 0.2, 0.3, 0.4]),
    np.array([[0.1, 0.2, 0.3],
              [0.4, 0.5, 0.6],
              [0.7, 0.8, 0.9],
              [1.0, 1.1, 1.2]]),
    np.array([0.1, 0.2, 0.3]),
    np.array([[0.1], [0.2], [0.3]]),
    np.array([0.1])
]

weights2 = [
    np.array([[0.5, 0.6, 0.7, 0.8],
              [0.1, 0.2, 0.3, 0.4]]),
    np.array([0.5, 0.6, 0.7, 0.8]),
    np.array([[0.2, 0.3, 0.4],
              [0.5, 0.6, 0.7],
              [0.8, 0.9, 1.0],
              [1.1, 1.2, 1.3]]),
    np.array([0.2, 0.3, 0.4]),
    np.array([[0.2], [0.3], [0.4]]),
    np.array([0.2])
]

dataset = [
    (weights1,
     weights2)
]

parent_fitnesses = [
    (2.0,
     3.0)
]

gp_instance = GP(
    max_tree_depth=3,
    population_size=10,
    num_generations=5,
    mutation_rate=0.1,
    crossover_rate=0.8,
    elite_rate=0.1,
    input_shape=(3,),
    output_size=3
)
gp_instance.dataset = dataset
gp_instance.parent_fitnesses = parent_fitnesses


population = gp_instance.generate_population()
print("Initial Population:")
for tree in population:
    print(tree)
    print("------------------")


mutated_tree = gp_instance.mutate_tree(population[0])
print("\nMutated Tree:")
print(mutated_tree)


offspring1, offspring2 = gp_instance.crossover_trees(population[0], population[1])
print("\nOffspring after Crossover:")
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)


fitness_scores = [gp_instance.fitness(tree) for tree in population]
print("\nFitness Scores:")
print(fitness_scores)


best_tree = gp_instance.gp(dataset, parent_fitnesses)
print("\nBest Tree after Evolution:")
print(best_tree)

Initial Population:
['-', ['Mean', 'W1', 'W2'], 0.5]
------------------
['1Point', ['*', ['Minus', 'W2', 'W2'], 1], ['Mean', 'W2', ['Sum', 'W1', 'W1']]]
------------------
['+', 'W2', 2]
------------------
['Mean', ['-', ['*', 'W2', 2], 1], ['1Point', 'W1', 'W2']]
------------------
['Minus', ['Sum', 'W1', 'W1'], ['*', ['Mean', 'W1', 'W2'], 1]]
------------------
['*', 'W1', 0.5]
------------------
['-', 'W1', 2]
------------------
['Mean', ['Sum', ['+', 'W2', 1], ['Mean', 'W2', 'W2']], ['*', 'W2', 2]]
------------------
['-', ['*', 'W1', 1], 2]
------------------
['-', 'W1', 1]
------------------

Mutated Tree:
['-', ['Minus', 'W2', 'W1'], 0.5]

Offspring after Crossover:
Offspring 1: ['-', 'W1', 0.5]
Offspring 2: ['1Point', ['*', ['Minus', 'W2', 'W2'], 1], ['Mean', 'W2', ['Sum', ['Mean', 'W1', 'W2'], 'W1']]]

Fitness Scores:
[0.3603125503389628, 0.7186208270095309, 0.578476513322277, 0.4243573667656133, 0.355291834610559, 0.7031274760103775, 1.416135636369286, 1.036298589242748, 0.25